# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [84]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import processingmm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
# set the parameters to run the script
directories = ['/media/elea/ssd/NPP_dev/test']
calib_directory = '/media/elea/ssd/NPP_dev/calib'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
parameter_set = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = False

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'no'

# do not specify unless you want to use a custom path for the PDDN models
PDDN_models_path = None

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550]

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz

# NB: processing time without PDDN (with the data on a SSD drive)
# 'no_viz': 0.39s
# 'default', save_pdf_figs False: 1.65s
# 'default', save_pdf_figs True: 2.67s
# 'full', save_pdf_figs False: 3.24s
# 'full', save_pdf_figs True: 5.94s
processing_mode = 'full'
save_pdf_figs = False

# define if the wavelenghts should be aligned before processing - and used for the computation
align_wls = True

parameters = processingmm.get_parameters(directories, calib_directory, wavelengths, parameter_set = parameter_set, 
                                PDDN_mode = PDDN_mode, PDDN_models_path = PDDN_models_path, 
                                processing_mode = processing_mode, run_all = run_all, 
                                save_pdf_figs = save_pdf_figs, align_wls = align_wls)

In [92]:
%%time
times = processingmm.batch_process_master(parameters)
times

processing without PDDN...


0it [00:00, ?it/s]

processing without PDDN done.
CPU times: user 3.39 ms, sys: 23 μs, total: 3.41 ms
Wall time: 3.09 ms


{'total': 0}

In [129]:
%%time
from processingmm.addons import visualization_lines

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
run_all = False
times = visualization_lines.batch_visualization(parameters, run_all)
times

CPU times: user 280 μs, sys: 1.02 ms, total: 1.3 ms
Wall time: 1.08 ms


0.0005217790603637695

In [131]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths
directories = ['/media/elea/ssd/NPP_dev/test']

PDDN_mode = 'both'
run_all = False
align_wavelengths.align_wavelengths(directories, PDDN_mode, run_all, [600])

Aligning wavelength: 600 nm...


100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 11554.56it/s]

Aligning wavelengths done.

